In [3]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 10.6 MB/s eta 0:00:0000:0100:01


In [2]:
import itertools
import os
import shutil

import numpy as np
from scipy.sparse import csr_matrix

import xgboost
from xgboost import testing as tm

In [3]:
Xrow = np.array([1, 2, 6, 8, 11, 14, 16, 17])
Xcol = np.array([0, 0, 1, 1,  2,  2,  3,  3])
X = csr_matrix((np.ones(shape=8), (Xrow, Xcol)), shape=(20, 4))
y = np.array([0.0, 1.0, 1.0, 0.0, 0.0,
              0.0, 1.0, 0.0, 1.0, 0.0,
              0.0, 1.0, 0.0, 0.0, 1.0,
              0.0, 1.0, 1.0, 0.0, 0.0])

group = np.array([5, 5, 5, 5], dtype=np.uint)
dtrain = xgboost.DMatrix(X, label=y)
dtrain.set_group(group)

params = {'eta': 1, 'tree_method': 'exact',
          'objective': 'rank:pairwise', 'eval_metric': ['auc', 'aucpr'],
          'max_depth': 1}
evals_result = {}
bst = xgboost.train(params, dtrain, 10, evals=[(dtrain, 'train')],
                    evals_result=evals_result)
auc_rec = evals_result['train']['auc']
assert all(p <= q for p, q in zip(auc_rec, auc_rec[1:]))
auc_rec = evals_result['train']['aucpr']
assert all(p <= q for p, q in zip(auc_rec, auc_rec[1:]))

[0]	train-auc:0.57500	train-aucpr:0.55000
[1]	train-auc:0.65000	train-aucpr:0.70000
[2]	train-auc:0.72500	train-aucpr:0.85000
[3]	train-auc:0.80000	train-aucpr:1.00000
[4]	train-auc:0.80000	train-aucpr:1.00000
[5]	train-auc:0.80000	train-aucpr:1.00000
[6]	train-auc:0.80000	train-aucpr:1.00000
[7]	train-auc:0.80000	train-aucpr:1.00000
[8]	train-auc:0.80000	train-aucpr:1.00000
[9]	train-auc:0.80000	train-aucpr:1.00000


In [7]:
X.toarray()

array([[0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [9]:
y

array([0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1.,
       1., 0., 0.])

In [8]:
# Step 1: Import libraries
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score, average_precision_score

# Step 2: Convert data to numpy arrays and reshape
X = np.array([[0,1,1], [1,1,0], [1,1,1]])
Y = np.array([['a', 'b', 'c'], ['a', 'c', 'b'], ['b', 'a', 'c']])
X = X.reshape(-1, 1)
Y = Y.reshape(-1, 3)
print(X)
# Step 3: Create group array
group = np.array([3,3,3])

# Step 4: Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Step 5: Define xgboost parameters
params = {
'objective': 'rank:pairwise',
'learning_rate': 0.1,
'max_depth': 3,
'n_estimators': 100,
'seed': 42
}

# Step 6: Create DMatrix object from train data and group array
dtrain = xgb.DMatrix(X_train, label=Y_train)
dtrain.set_group(group)

# Step 7: Train xgboost model
model = xgb.train(params, dtrain)

# Step 8: Create DMatrix object from test data and group array
dtest = xgb.DMatrix(X_test)
dtest.set_group(group)

# Step 9: Predict rankings for test data
preds = model.predict(dtest, output_margin=True)
preds = preds.reshape(-1, 3)

# Step 10: Evaluate performance using ranking metrics
ndcg = ndcg_score(Y_test, preds)
map = average_precision_score(Y_test.ravel(), preds.ravel())
print(f'NDCG score: {ndcg}')
print(f'MAP score: {map}')

[[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]]


ValueError: Found input variables with inconsistent numbers of samples: [9, 3]

In [1]:
X = [[0,1,1], [1,1,0], [1,1,1]]
Y = [['a', 'b', 'c'], ['a', 'c', 'b'], ['b', 'a', 'c']]

In [6]:
from sklearn.preprocessing import LabelEncoder

# label encode the Y data
le = LabelEncoder()
Y_encoded = le.fit_transform([y for y_list in Y for y in y_list])

# create an XGBoost model and train it
import xgboost as xgb

# flatten the X data to a 2D array
X_flat = [x for x_list in X for x in x_list]
print(X_flat)
print(Y_encoded)
# set up the XGBoost model
dtrain = xgb.DMatrix(X_flat, label=Y_encoded)
params = {
    'objective': 'rank:pairwise',

}


# train the model
model = xgb.train(params, dtrain, num_rounds)

# make predictions
dtest = xgb.DMatrix(X_flat)
preds = model.predict(dtest)

[0, 1, 1, 1, 1, 0, 1, 1, 1]
[0 1 2 0 2 1 1 0 2]


ValueError: ('Expecting 2 dimensional numpy.ndarray, got: ', (9,))